In [11]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split, dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dir_path = r"C:\Users\orian\OneDrive\שולחן העבודה\Coding Files 2\Datasets\chat gpt reviews"
dataset = pd.read_csv(dir_path+r"\ChatGPT_Reviews.csv")

train_len = round(len(dataset)*0.9)
train_ds_csv, test_ds_csv = random_split(
    dataset=dataset,lengths=[train_len, len(dataset)-train_len])

train_ds = DataLoader(train_ds_csv, shuffle= True)
print(train_ds)
    